# AFAD

In [1]:
import time
from bs4 import BeautifulSoup
import requests
import json

AFAD_URL = "https://deprem.afad.gov.tr/last-earthquakes.html"

def scrape_afad_earthquake_data(output_file, max_pages=1000):
    """
    AFAD web sitesinden deprem verilerini scrape eder ve JSON dosyasına kaydeder.
    Çok sayıda sayfa dolaşır.
    """
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        earthquake_data = []

        for page in range(1, max_pages + 1):
            print(f"Sayfa {page} işleniyor...")
            # Sayfa parametresini URL'ye ekle
            response = requests.get(f"{AFAD_URL}?page={page}", headers=headers)
            if response.status_code == 404:  # Eğer daha fazla sayfa yoksa döngüyü kır
                print("Tüm sayfalar işlendi.")
                break

            soup = BeautifulSoup(response.content, "html.parser")
            table = soup.find("table")
            if not table:
                raise ValueError("Tablo bulunamadı. Tablo yapısını kontrol edin.")

            rows = table.find_all("tr")[1:]  # Başlık satırını atla

            for row in rows:
                cols = row.find_all("td")
                if len(cols) >= 6:
                    earthquake = {
                        "date": cols[0].text.strip(),
                        "latitude": cols[1].text.strip(),
                        "longitude": cols[2].text.strip(),
                        "depth": cols[3].text.strip(),
                        "magnitude": cols[4].text.strip(),
                        "location": cols[5].text.strip(),
                    }
                    earthquake_data.append(earthquake)

            # Belleği korumak için verileri ara ara dosyaya yaz
            if page % 10 == 0 or page == max_pages:
                with open(output_file, "a") as file:
                    json.dump(earthquake_data, file, indent=4)
                earthquake_data.clear()  # Geçici belleği boşalt

            # Sunucuyu aşırı yüklememek için bekle
            time.sleep(1)

        print(f"AFAD verileri '{output_file}' dosyasına kaydedildi.")
    except Exception as e:
        print(f"Veri çekme başarısız oldu: {e}")

# Örnek kullanım
if __name__ == "__main__":
    scrape_afad_earthquake_data("afad_more.json", max_pages=1000)


Sayfa 1 işleniyor...
Sayfa 2 işleniyor...
Sayfa 3 işleniyor...
Sayfa 4 işleniyor...
Sayfa 5 işleniyor...
Sayfa 6 işleniyor...
Sayfa 7 işleniyor...
Sayfa 8 işleniyor...
Sayfa 9 işleniyor...
Sayfa 10 işleniyor...
Sayfa 11 işleniyor...
Sayfa 12 işleniyor...
Sayfa 13 işleniyor...
Sayfa 14 işleniyor...
Sayfa 15 işleniyor...
Sayfa 16 işleniyor...
Sayfa 17 işleniyor...
Sayfa 18 işleniyor...
Sayfa 19 işleniyor...
Sayfa 20 işleniyor...
Sayfa 21 işleniyor...
Sayfa 22 işleniyor...
Sayfa 23 işleniyor...
Sayfa 24 işleniyor...
Sayfa 25 işleniyor...
Sayfa 26 işleniyor...
Sayfa 27 işleniyor...
Sayfa 28 işleniyor...
Sayfa 29 işleniyor...
Sayfa 30 işleniyor...
Sayfa 31 işleniyor...
Sayfa 32 işleniyor...
Sayfa 33 işleniyor...
Sayfa 34 işleniyor...
Sayfa 35 işleniyor...
Sayfa 36 işleniyor...
Sayfa 37 işleniyor...
Sayfa 38 işleniyor...
Sayfa 39 işleniyor...
Sayfa 40 işleniyor...
Sayfa 41 işleniyor...
Sayfa 42 işleniyor...
Sayfa 43 işleniyor...
Sayfa 44 işleniyor...
Sayfa 45 işleniyor...
Sayfa 46 işleniyor.

# KANDİLLİ

In [3]:
import time
from bs4 import BeautifulSoup
import requests
import json

# Kandilli Rasathanesi'nin son depremler URL'si
KANDILLI_URL = "http://www.koeri.boun.edu.tr/scripts/lst0.asp"

def scrape_kandilli_earthquake_data(output_file, max_entries=1000000):
    """
    Kandilli Rasathanesi web sitesinden deprem verilerini scrape eder ve JSON dosyasına kaydeder.
    Daha fazla veri çekmek için döngü eklenmiştir.
    """
    try:
        earthquake_data = []
        current_entries = 0

        while current_entries < max_entries:
            print(f"Veri çekiliyor, toplam giriş: {current_entries}")

            # Web sitesine GET isteği gönder
            response = requests.get(KANDILLI_URL)
            response.raise_for_status()

            # HTML içeriğini ayrıştır
            soup = BeautifulSoup(response.content, "html.parser")
            pre_tag = soup.find("pre")
            if not pre_tag:
                raise ValueError("Tablo verisi bulunamadı. Yapıyı kontrol edin.")

            # Verileri satırlara böl
            lines = pre_tag.text.strip().split("\n")[6:]  # İlk 6 satır başlık bilgisi
            if not lines:
                print("Yeni veri bulunamadı.")
                break

            # Satırları ayrıştır
            for line in lines:
                columns = line.split()
                if len(columns) >= 9:
                    earthquake = {
                        "date": f"{columns[0]} {columns[1]}",
                        "latitude": columns[2],
                        "longitude": columns[3],
                        "depth_km": columns[4],
                        "magnitude": columns[6],
                        "location": " ".join(columns[8:]),
                    }
                    earthquake_data.append(earthquake)
                    current_entries += 1

                    if current_entries >= max_entries:
                        break

            # Veriler ara ara dosyaya kaydedilir
            if current_entries % 100 == 0 or current_entries >= max_entries:
                with open(output_file, "a") as file:
                    json.dump(earthquake_data, file, indent=4, ensure_ascii=False)
                earthquake_data.clear()  # Belleği boşalt
                print(f"{current_entries} giriş kaydedildi.")

            # Bekleme süresi ekle (sunucunun IP engellemesini önlemek için)
            time.sleep(5)

        print(f"Tüm veriler '{output_file}' dosyasına kaydedildi.")
    except Exception as e:
        print(f"Veri çekme başarısız oldu: {e}")

# Örnek kullanım
if __name__ == "__main__":
    scrape_kandilli_earthquake_data("kandilli_more_data.json", max_entries=1000000)


Veri çekiliyor, toplam giriş: 0
500 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 500
1000 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 1000
1500 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 1500
2000 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 2000
2500 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 2500
3000 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 3000
3500 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 3500
4000 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 4000
4500 giriş kaydedildi.
Veri çekiliyor, toplam giriş: 4500
5000 giriş kaydedildi.


KeyboardInterrupt: 

kandilli den veri çektim sadece bir yerden sonra kodu durdurdum

# USGS

In [1]:
import requests
import json
from datetime import datetime, timedelta

# USGS API endpoint
USGS_API_URL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

def fetch_large_earthquake_data(start_date, end_date, min_magnitude, max_magnitude, output_file, days_per_request=7):
    """
    USGS API'den çok fazla deprem verisini parça parça çeker ve JSON dosyasına ekler.
    """
    try:
        current_start_date = start_date

        while current_start_date < end_date:
            current_end_date = min(current_start_date + timedelta(days=days_per_request), end_date)

            print(f"{current_start_date} ile {current_end_date} arası veri çekiliyor...")
            params = {
                "format": "geojson",
                "starttime": current_start_date.strftime("%Y-%m-%d"),
                "endtime": current_end_date.strftime("%Y-%m-%d"),
                "minmagnitude": min_magnitude,
                "maxmagnitude": max_magnitude
            }

            # API isteği gönder
            response = requests.get(USGS_API_URL, params=params)
            response.raise_for_status()  # Hata kontrolü

            # Yanıtı JSON formatında işleme
            data = response.json()
            earthquakes = data.get("features", [])

            if not earthquakes:
                print(f"{current_start_date} ile {current_end_date} arasında deprem bulunamadı.")
            else:
                # Verileri JSON dosyasına ekle
                with open(output_file, "a") as file:
                    for earthquake in earthquakes:
                        json.dump(earthquake, file, indent=4)
                        file.write("\n")  # Her deprem arasında yeni satır ekle

                print(f"{len(earthquakes)} deprem kaydedildi.")

            # Zaman aralığını ilerlet
            current_start_date = current_end_date

    except requests.exceptions.RequestException as e:
        print(f"API isteği başarısız oldu: {e}")
    except Exception as e:
        print(f"Beklenmeyen bir hata oluştu: {e}")

# Örnek kullanım
if __name__ == "__main__":
    # Başlangıç ve bitiş tarihlerini tanımla (20 yıl geriye kadar çekim yapılabilir)
    start_date = datetime(2000, 1, 1)  # 1 Ocak 2000
    end_date = datetime.utcnow()       # Şu anki tarih ve saat

    fetch_large_earthquake_data(
        start_date=start_date,
        end_date=end_date,
        min_magnitude=5.0,  # Minimum büyüklük
        max_magnitude=10.0, # Maksimum büyüklük
        output_file="large_earthquake_data.json",
        days_per_request=30  # Her istekte 30 günlük veri çek
    )


2000-01-01 00:00:00 ile 2000-01-31 00:00:00 arası veri çekiliyor...
105 deprem kaydedildi.
2000-01-31 00:00:00 ile 2000-03-01 00:00:00 arası veri çekiliyor...
96 deprem kaydedildi.
2000-03-01 00:00:00 ile 2000-03-31 00:00:00 arası veri çekiliyor...
91 deprem kaydedildi.
2000-03-31 00:00:00 ile 2000-04-30 00:00:00 arası veri çekiliyor...
83 deprem kaydedildi.
2000-04-30 00:00:00 ile 2000-05-30 00:00:00 arası veri çekiliyor...
95 deprem kaydedildi.
2000-05-30 00:00:00 ile 2000-06-29 00:00:00 arası veri çekiliyor...
151 deprem kaydedildi.
2000-06-29 00:00:00 ile 2000-07-29 00:00:00 arası veri çekiliyor...
145 deprem kaydedildi.
2000-07-29 00:00:00 ile 2000-08-28 00:00:00 arası veri çekiliyor...
130 deprem kaydedildi.
2000-08-28 00:00:00 ile 2000-09-27 00:00:00 arası veri çekiliyor...
105 deprem kaydedildi.
2000-09-27 00:00:00 ile 2000-10-27 00:00:00 arası veri çekiliyor...
103 deprem kaydedildi.
2000-10-27 00:00:00 ile 2000-11-26 00:00:00 arası veri çekiliyor...
253 deprem kaydedildi.
200

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("warcoder/earthquake-dataset")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'

In [6]:
pip install kaggle


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Kaggle API'yi başlat
api = KaggleApi()
api.authenticate()

# Veri seti ID'sini ve hedef dizini ayarla
dataset = "warcoder/earthquake-dataset"  # Kaggle veri seti ID'si
output_dir = "earthquake_data"

# İndirilecek dizini oluştur
os.makedirs(output_dir, exist_ok=True)

# Kaggle veri setini indir
print(f"Downloading dataset: {dataset}")
api.dataset_download_files(dataset, path=output_dir, unzip=True)

print(f"Dataset downloaded to: {output_dir}")


OSError: Could not find kaggle.json. Make sure it's located in /Users/gokcesoylu/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [2]:
import os

# Kaggle kimlik bilgilerini ortam değişkenleri olarak ayarla
os.environ["KAGGLE_USERNAME"] = "gkesoylu"
os.environ["KAGGLE_KEY"] = "ee7a3e9d9f6e558e1987a9a9fd9ecda8"


In [3]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Kaggle kimlik bilgilerini ortam değişkenleri olarak ayarla
os.environ["KAGGLE_USERNAME"] = "gkesoylu"
os.environ["KAGGLE_KEY"] = "ee7a3e9d9f6e558e1987a9a9fd9ecda8"

# Kaggle API'yi başlat
api = KaggleApi()
api.authenticate()

# Veri seti ID'sini ve hedef dizini ayarla
dataset = "warcoder/earthquake-dataset"  # Kaggle veri seti ID'si
output_dir = "earthquake_data"

# İndirilecek dizini oluştur
os.makedirs(output_dir, exist_ok=True)

# Kaggle veri setini indir
print(f"Downloading dataset: {dataset}")
api.dataset_download_files(dataset, path=output_dir, unzip=True)

print(f"Dataset downloaded to: {output_dir}")


Dataset URL: https://www.kaggle.com/datasets/warcoder/earthquake-dataset
Dataset downloaded to: earthquake_data
